### 1) Import Packages

In [67]:
import pypsa

import os
import matplotlib.pyplot as plt
import numpy as npim
import pandas as pd
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

In [68]:
# Create a new PyPSA network
network = pypsa.Network()
network.set_snapshots(range(1))  # Solve for a year 365*24

In [69]:
type(network)

pypsa.components.Network

In [70]:
# Read excel file which contains Non renewable generators data
bus=pd.read_excel('data_exercise.xlsx',sheet_name='buses',header=0)

In [71]:
bus

,bus,v_nom,carrier,x,y
0,north_de,380,AC,53.558119,9.598188
1,east_de,380,AC,51.076718,13.607779
2,west_de,380,AC,50.981593,6.406648


In [72]:
for index, row in bus.iterrows():
    network.add(
        "Bus",
        name=row['bus'],
        v_nom=row['v_nom'],
        carrier=row['carrier'],
        x=row['x'],
        y=row['y']
    )

In [73]:
network.buses

,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
north_de,380.0,,53.558119,9.598188,AC,,1.0,0.0,inf,PQ,,
east_de,380.0,,51.076718,13.607779,AC,,1.0,0.0,inf,PQ,,
west_de,380.0,,50.981593,6.406648,AC,,1.0,0.0,inf,PQ,,


In [74]:
# Read excel file which contains Non renewable generators data
generators=pd.read_excel('data_exercise.xlsx',sheet_name='generators')

In [75]:
generators

,generator,p_nom,p_nom_max,bus,p_nom_extendable,marginal_cost,lifetime,efficiency,start_up_cost,shut_down_cost,min_up_time,min_down_time,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
0,gen_no1,15,NaN,north_de,False,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gen_no2,16,NaN,north_de,False,15,NaN,NaN,NaN,NaN,4.0,3.0,0.54,0.75,NaN,NaN
2,gen_no3,9,NaN,north_de,False,30,NaN,NaN,NaN,NaN,2.0,2.0,0.75,0.75,NaN,NaN
3,gen_ost1,10,NaN,east_de,False,15,NaN,NaN,NaN,NaN,2.0,2.0,0.75,0.75,NaN,NaN
4,gen_ost2,9,NaN,east_de,False,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,gen_ost3,5,NaN,east_de,False,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,gen_west1,30,NaN,west_de,False,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,gen_west2,10,NaN,west_de,False,40,NaN,NaN,NaN,NaN,2.0,2.0,0.75,0.75,NaN,NaN
8,gen_west3,15,NaN,west_de,False,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
network.madd(
    "Generator",
    names= generators.generator,
    bus=list(generators.bus),
    carrier=list(generators.generator),
    p_nom=list(generators.p_nom),
    #p_nom_max=list(generators.p_nom_max),  # MW
    p_nom_extendable=list(generators.p_nom_extendable),
    marginal_cost=list(generators.marginal_cost),
)
network.generators

C:\Users\AJESH\AppData\Local\Temp\ipykernel_9788\3897436386.py:1: DeprecatedWarning:

madd is deprecated as of 0.31 and will be removed in 1.0. Use `n.add` as a drop-in replacement instead.



,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
gen_no1,north_de,PQ,,15.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no2,north_de,PQ,,16.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no3,north_de,PQ,,9.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_ost1,east_de,PQ,,10.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_ost2,east_de,PQ,,9.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_ost3,east_de,PQ,,5.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_west1,west_de,PQ,,30.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_west2,west_de,PQ,,10.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_west3,west_de,PQ,,15.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [77]:
network.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
gen_no1,north_de,PQ,,15.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no2,north_de,PQ,,16.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_no3,north_de,PQ,,9.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_ost1,east_de,PQ,,10.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_ost2,east_de,PQ,,9.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_ost3,east_de,PQ,,5.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_west1,west_de,PQ,,30.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_west2,west_de,PQ,,10.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gen_west3,west_de,PQ,,15.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [78]:
# Read excel file which contains Non renewable generators data
load=pd.read_excel('data_exercise.xlsx',sheet_name='load',header=0)

In [79]:
load

,bus,p_set
0,north_de,10
1,east_de,10
2,west_de,40


In [80]:
network.madd(
    "Load",
    names= load.bus,
    bus=list(load.bus),
    p_set=list(load.p_set),
    #p_nom=list(load.p_nom),
    #p_nom_max=list(load.p_nom_max),  # MW
    #p_nom_extendable=list(load.p_nom_extendable),
    #marginal_cost=list(load.marginal_cost),
)
network.loads

C:\Users\AJESH\AppData\Local\Temp\ipykernel_9788\2893457690.py:1: DeprecatedWarning:

madd is deprecated as of 0.31 and will be removed in 1.0. Use `n.add` as a drop-in replacement instead.



,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
north_de,north_de,,,10.0,0.0,-1.0,True
east_de,east_de,,,10.0,0.0,-1.0,True
west_de,west_de,,,40.0,0.0,-1.0,True


In [81]:
network.loads

,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
north_de,north_de,,,10.0,0.0,-1.0,True
east_de,east_de,,,10.0,0.0,-1.0,True
west_de,west_de,,,40.0,0.0,-1.0,True


In [82]:
# Read excel file which contains Non renewable generators data
lines=pd.read_excel('data_exercise.xlsx',sheet_name='lines',header=0)

In [83]:
lines.head()

,country_1,country_2,cap_MW
0,north_de,east_de,500
1,east_de,west_de,500
2,west_de,north_de,500


In [84]:
# Add line connection between the buses
for index, row in lines.iterrows():
    network.add(
        "Line",
        "{}_{}".format(row['country_1'],row['country_2']),
        bus0=row['country_1'],
        bus1=row['country_2'],
        s_nom=row['cap_MW'],
        x=1,
    )

In [85]:
network.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
north_de_east_de,north_de,east_de,,1.0,0.0,0.0,0.0,500.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
east_de_west_de,east_de,west_de,,1.0,0.0,0.0,0.0,500.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
west_de_north_de,west_de,north_de,,1.0,0.0,0.0,0.0,500.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
network

PyPSA Network
Components:
 - Bus: 3
 - Generator: 9
 - Line: 3
 - Load: 3
Snapshots: 1

In [87]:
network.optimize(solver_name='highs')

Index(['gen_no1', 'gen_no2', 'gen_no3', 'gen_ost1', 'gen_ost2', 'gen_ost3',
       'gen_west1', 'gen_west2', 'gen_west3'],
      dtype='object', name='Generator')
Index(['north_de_east_de', 'east_de_west_de', 'west_de_north_de'], dtype='object', name='Line')
Index(['north_de_east_de', 'east_de_west_de', 'west_de_north_de'], dtype='object', name='Line')
Index(['north_de', 'east_de', 'west_de'], dtype='object', name='Bus')
Index(['gen_no1', 'gen_no2', 'gen_no3', 'gen_ost1', 'gen_ost2', 'gen_ost3',
       'gen_west1', 'gen_west2', 'gen_west3'],
      dtype='object', name='Generator')
Index(['north_de_east_de', 'east_de_west_de', 'west_de_north_de'], dtype='object', name='Line')
Index(['north_de_east_de', 'east_de_west_de', 'west_de_north_de'], dtype='object', name='Line')
Index(['north_de', 'east_de', 'west_de'], dtype='object', name='Bus')
C:\Users\AJESH\AppData\Roaming\Python\Python312\site-packages\linopy\common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned

('ok', 'optimal')

In [88]:
network.buses_t.marginal_price

Bus,north_de,east_de,west_de
snapshot,,,
0,25.0,25.0,25.0


In [89]:
network.generators_t.p


Generator,gen_no1,gen_no2,gen_no3,gen_ost1,gen_ost2,gen_ost3,gen_west1,gen_west2,gen_west3
snapshot,,,,,,,,,
0,15.0,16.0,-0.0,10.0,9.0,-0.0,10.0,-0.0,-0.0


In [90]:
clearing_price_redisp = network.buses_t.marginal_price.iloc[0].max()
clearing_price_redisp

25.0